In [1]:
import pandas as pd

# 18 - 23

In [2]:
df_18 = pd.read_excel("Consommation  par mois  2018 2023.xlsx", sheet_name='CD')

In [3]:
df_18.head()

,Consommation en MWh,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147
0,Ventes ONEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Partenaire,Contrat,Activité,Secteur,VENTES ONEE En MWh,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,2018,2018.0000,2018.0000,2018.0000,2018.0000,2018.0000,...,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000
3,NaN,NaN,NaN,NaN,01.2018,2.2018,3.2018,4.2018,5.2018,6.2018,...,3.2023,4.2023,5.2023,6.2023,7.2023,8.2023,9.2023,10.2023,11.2023,12.2023
4,ACWA POWER KHALLADI,6434280,"PRODUCTION ET DISTTRIBUTION D'ELECTRICITE, DE ...",PROD. ET DIST. D'ÉLECTRICITÉ,NaN,NaN,NaN,NaN,NaN,NaN,...,66.4290,58.6520,80.6740,20.9440,94.7210,83.3140,62.7000,25.5750,101.0350,50.4350


In [4]:
for i in range(len(df_18.iloc[1])):
    value = df_18.iloc[1, i]
    if pd.notna(value):
        print(i, value)

0 Partenaire 
1 Contrat
2 Activité 
3 Secteur 
4 VENTES ONEE En MWh
76 Consommation en MWh


## Ventes

In [5]:
df_18_ventes = df_18.iloc[1:, :76]

In [6]:
df_18_ventes.shape

(172, 76)

In [7]:
new_cols = list(df_18_ventes.iloc[0, :4].values) + list(df_18_ventes.iloc[2, 4:].values)

In [8]:
def clean_col_name(col_name):
    try:
        col_name = float(col_name)
        col_name = f"{col_name:.4f}"
    except:
        pass
    return col_name.strip().lower().replace(' ', '_')

new_cols = [clean_col_name(col) for col in new_cols]

In [9]:
df_18_ventes.columns = new_cols

In [10]:
df_18_ventes = df_18_ventes.iloc[3:, :].reset_index(drop=True)

In [11]:
df_18_ventes = df_18_ventes.melt(
    id_vars=['partenaire', 'contrat', 'activité', 'secteur'],
    var_name='date',
    value_name='ventes'  
)

In [12]:
def get_year_month(row):
    date_str = row["date"]
    try:
        parts = date_str.split('.')
        year = int(parts[1])
        month = int(parts[0])
        row["mois"] = month
        row["année"] = year
        del row["date"]
        return row
    except:
        print("error")

In [13]:
df_18_ventes = df_18_ventes.apply(get_year_month, axis=1)

In [14]:
df_18_ventes.shape

(12168, 7)

## Consomation

In [15]:
df_18_consomation = pd.concat([df_18.iloc[4:, :4], df_18.iloc[4:, 76:]], axis = 1)

In [16]:
df_18_consomation.shape

(169, 76)

In [17]:
df_18_consomation.columns = new_cols

In [18]:
df_18_consomation.reset_index(drop=True, inplace=True)

In [19]:
df_18_consomation = df_18_consomation.melt(
    id_vars=['partenaire', 'contrat', 'activité', 'secteur'],
    var_name='date',
    value_name='consomation'  
)

In [20]:
df_18_consomation = df_18_consomation.apply(get_year_month, axis=1)

In [21]:
df_18_consomation.shape

(12168, 7)

### Merging

In [22]:
df_18_merged = df_18_ventes.merge(
    df_18_consomation[['contrat', 'mois', 'année', 'consomation']], 
    on=['contrat', 'mois', 'année'],
    how='left'
)

In [23]:
len(df_18_ventes["contrat"].unique())

169

### tests

In [24]:
df_18_merged[df_18_merged['contrat']==6434280]["consomation"].sum()

3392.4220000000005

In [25]:
df_18_merged.dtypes

partenaire      object
contrat          int64
activité        object
secteur         object
ventes         float64
mois             int64
année            int64
consomation     object
dtype: object

In [26]:
for el in enumerate(df_18_merged["consomation"]):
    try:
        float(el[1])
    except:
        print(el)

(14, '²')


In [27]:
df_18_merged["consomation"] = pd.to_numeric(df_18_merged["consomation"], errors='coerce')

In [28]:
df_18_merged.dtypes

partenaire      object
contrat          int64
activité        object
secteur         object
ventes         float64
mois             int64
année            int64
consomation    float64
dtype: object

In [31]:
df_18_merged.isna().sum()

partenaire     0
contrat        0
activité       0
secteur        0
ventes         0
mois           0
année          0
consomation    0
dtype: int64

In [30]:
df_18_merged.fillna(0, inplace=True)

In [33]:
df_18_merged.to_csv("cd_data_2018.csv", index=False)

# 13 - 17

In [35]:
df_13 = pd.read_excel("Données GC 2013 -2017.xlsx", header=None)

In [36]:
df_13.head()

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,NaN,NaN,NaN,NaN,NaN,NaN,Ventes ONEE en KWh,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Catégorie,Contrat,Partenaire,NaN,Activité,Secteur,2013,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Contrat,NaN,NaN,1,2.0,3.0,4.0,...,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
3,C. Directs,217004,Client 56,106,METALLURGIE,MÉTALLURGIE PLOMB ZINC ÉTAIN,209146,204069.0,191601.0,147204.0,...,79020.0,70720.0,78805.0,89202.0,85309.0,88513.0,82631.0,77158.0,72639.0,55590.0
4,C. Directs,217005,Client 7,34,FABRICATION D'AUTRES PRODUITS MINERAUX NON MET...,FABRICATION DE CIMENT,10796605,9369370.0,8627980.0,10865265.0,...,5876300.0,6882412.0,4004403.0,2201659.0,6248838.0,6474130.0,5740938.0,6517663.0,6036143.0,4455282.0


In [37]:
df_13 = df_13[df_13[0] != "Distributeurs"]
df_13.drop(columns = [0, 3], inplace=True)


In [38]:
for i in range(len(df_13.iloc[0])):
    value = df_13.iloc[0, i]
    if pd.notna(value):
        print(i, value)

4 Ventes ONEE en KWh
64 Achat en KWh auprés  des producteurs privés 
124 Consommation en KWh 


In [39]:
new_cols_13 = ['contrat', 'partenaire', 'activité', 'secteur']
new_cols_13 += [f"{j}.{i}" for i in range(2013, 2018) for j in range(1,13)]

In [40]:
df_13 = df_13.iloc[3:]
df_13.dropna(subset=[1,2,4,5], inplace=True)

## ventes

In [41]:
df_13_ventes = df_13.iloc[:, :64]

In [42]:
df_13_ventes.shape

(142, 64)

In [43]:
df_13_ventes.columns = new_cols_13

In [44]:
df_13_ventes = df_13_ventes.melt(
    id_vars=['contrat', 'partenaire', 'activité', 'secteur'],
    var_name='date',
    value_name='ventes'  
)

In [45]:
df_13_ventes = df_13_ventes.apply(get_year_month, axis=1)

## PV

In [160]:
df_13_pv = pd.concat([df_13.iloc[:, :4],df_13.iloc[:, 64:124]], axis=1)

In [187]:
df_13_pv.shape

(136, 64)

## Consomation

In [46]:
df_13_consomation = pd.concat([df_13.iloc[:, :4],df_13.iloc[:, 124:]], axis=1)

In [47]:
df_13_consomation.shape

(142, 64)

In [48]:
df_13_consomation.columns = new_cols_13

In [49]:
df_13_consomation = df_13_consomation.melt(
    id_vars=['contrat', 'partenaire', 'activité', 'secteur'],
    var_name='date',
    value_name='consomation'  
)

In [50]:
df_13_consomation = df_13_consomation.apply(get_year_month, axis=1)

### merging

In [51]:
df_13_merged = df_13_ventes.merge(
    df_13_consomation[['contrat', 'mois', 'année', 'consomation']], 
    on=['contrat', 'mois', 'année'],
    how='left'
)

In [52]:
df_13_merged["ventes"] = df_13_merged["ventes"]/1000
df_13_merged["consomation"] = df_13_merged["consomation"]/1000

### testing

In [53]:
df_13_merged[df_13_merged['contrat']==217048]["ventes"].sum()

np.float64(10790.541000000001)

In [ ]:
df_13_merged.isna().sum()

In [55]:
df_13_merged.fillna(0, inplace=True)

In [56]:
df_13_merged.dtypes

contrat          int64
partenaire      object
activité        object
secteur         object
ventes         float64
mois             int64
année            int64
consomation    float64
dtype: object

# Merge

In [59]:
column_order =['partenaire', 'contrat', 'activité', 'secteur', 'année', 'mois', 'ventes', 'consomation']

In [60]:
df_13_merged = df_13_merged[column_order]
df_18_merged = df_18_merged[column_order]

In [102]:
df_18_merged.columns

Index(['partenaire', 'contrat', 'activité', 'secteur', 'année', 'mois',
       'ventes', 'consomation'],
      dtype='object')

In [104]:
import pandas as pd

# columns from df2 you want to repeat
repeat_cols = ['partenaire', 'activité', 'secteur', 'contrat']

# get the unique combos of the repeat columns for each contrat in df2
df2_keys = df_18_merged[repeat_cols].drop_duplicates()

# join df1 to those keys on contrat -> creates new rows with repeated fields
new_rows = df_13_merged[['contrat', 'année', 'mois', 'ventes', 'consomation']].merge(df2_keys, on='contrat', how='inner')

# reorder columns to match df2 (optional but tidy)
new_rows = new_rows[df_18_merged.columns]

# append to df2
df_final = pd.concat([df_18_merged, new_rows], ignore_index=True)

In [118]:
df_final = df_final.sort_values(
    by=['contrat', 'année', 'mois'],
    ascending=[True, True, True]
).reset_index(drop=True)


In [120]:
df_final.rename(columns={'consomation': 'consommation', "activité": "activite", "année": "annee"}, inplace=True)

In [123]:
df_final.isna().sum()

partenaire      0
contrat         0
activite        0
secteur         0
annee           0
mois            0
ventes          0
consommation    0
dtype: int64

In [124]:
df_final.dtypes

partenaire       object
contrat           int64
activite         object
secteur          object
annee             int64
mois              int64
ventes          float64
consommation    float64
dtype: object

In [38]:
import pandas as pd
df_final = pd.read_csv("cd_data_2013_2023.csv")

In [39]:
# --- config ---
YEARS_TO_ENSURE = range(2013, 2018)   # 2013–2017
MONTHS = range(1, 13)                  # 1–12

# Columns that are numeric we want to zero-fill for new rows
num_cols = ['ventes', 'consommation']

# Figure out which columns should be copied from an existing row
all_cols = list(df_final.columns)
meta_cols = [c for c in all_cols if c not in (['annee', 'mois'] + num_cols)]
# 'contrat' is in meta_cols right now, but we'll set it explicitly per-row

new_rows = []

for contrat in df_final['contrat'].unique():
    df_c = df_final[df_final['contrat'] == contrat]

    # If any of the required years are missing, we will add ALL 12 months for those years
    have_years = set(df_c['annee'].unique())
    missing_years = [y for y in YEARS_TO_ENSURE if y not in have_years]
    if not missing_years:
        continue  # this contrat already covers 2013–2017

    # Build a "template" of values to copy from an existing row for this contrat
    # (take the first non-null value found for each column)
    template = {}
    for col in meta_cols:
        if col == 'contrat':
            continue
        ser = df_c[col].dropna()
        template[col] = ser.iloc[0] if len(ser) else None

    # Create the missing rows (12 months per missing year) with zeros for ventes/consommation
    for y in missing_years:
        for m in MONTHS:
            row = {
                'contrat': contrat,
                'annee': y,
                'mois': m,
                'ventes': 0,
                'consommation': 0,
            }
            # copy the rest of the columns from the template
            for col in meta_cols:
                if col not in row and col != 'contrat':
                    row[col] = template.get(col, None)
            new_rows.append(row)

# Append the newly created rows and (optionally) sort
if new_rows:
    df_final = pd.concat([df_final, pd.DataFrame(new_rows)], ignore_index=True)

# Optional: sort for readability
sort_cols = [c for c in ['contrat', 'annee', 'mois'] if c in df_final.columns]
if sort_cols:
    df_final = df_final.sort_values(sort_cols).reset_index(drop=True)

In [40]:
len(df_final)

22308

In [42]:
df_final.to_csv("cd_data_2013_2023.csv", index=False)

In [43]:
import sqlite3

# Path to your database file
db_path = "cd_database_2013_2023.db"

# Connect to the database (creates file if it doesn’t exist)
conn = sqlite3.connect(db_path)

# Save the DataFrame as a new table
df_final.to_sql("cd", conn, if_exists="replace", index=False)

# Close the connection
conn.close()
